In [64]:
import pandas as pd
from datetime import datetime

excel_path = '/Users/zeynepdundar/Downloads/INDR491_UPDATED.xlsx'
df = pd.read_excel (excel_path)


In [91]:
threshold_values = 5
interval_freq = "180min"
interval_offset = "2s"
number_of_states = 4

threshold_state_1 = 0.3
threshold_state_2 = 0.65
threshold_state_3 = 0.95


df_new = df.copy()
timestamp = 1553367060

# make a plain dataframe containing only date and value columns
df_new_sade = pd.concat([df_new["datetime"], df_new["Value Updated"]], axis=1)
df_new_sade


,datetime,Value Updated
0,2021-11-16 20:36:31.838,0.003
1,2021-11-16 20:37:01.838,0.003
2,2021-11-16 20:37:31.838,0.003
3,2021-11-16 20:38:01.838,0.003
4,2021-11-16 20:38:31.838,0.000
...,...,...
1048570,2022-11-15 22:41:31.838,0.261
1048571,2022-11-15 22:42:01.838,0.264
1048572,2022-11-15 22:42:31.838,0.267
1048573,2022-11-15 22:43:01.838,0.264


In [66]:
#create groups of dataframe by time intervals and select the rows which are bigger than the _for_values value
numerator = df_new_sade.groupby(pd.Grouper(key='datetime', offset=interval_offset, freq=interval_freq, label='right')).apply(lambda x: x[x["Value Updated"]>threshold_values]["Value Updated"].sum())
denominator = df_new_sade.groupby(pd.Grouper(key='datetime', offset=interval_offset, freq=interval_freq, label='right')).apply(lambda x: x["Value Updated"].sum())

#calculate proportions for every period
proportions = numerator/denominator
proportions 

datetime
2021-11-16 21:00:02    0.0
2021-11-17 00:00:02    0.0
2021-11-17 03:00:02    0.0
2021-11-17 06:00:02    0.0
2021-11-17 09:00:02    0.0
                      ... 
2022-11-15 12:00:02    0.0
2022-11-15 15:00:02    0.0
2022-11-15 18:00:02    0.0
2022-11-15 21:00:02    0.0
2022-11-16 00:00:02    0.0
Freq: 180T, Length: 2914, dtype: float64

In [92]:
# check the nonzero values 
# numerator
# denominator
# proportions
(numerator[numerator != 0])
#(proportions[proportions != 0])

datetime
2021-12-11 21:00:02     1462.212
2021-12-12 00:00:02    11054.448
2021-12-12 03:00:02    11245.824
2021-12-12 06:00:02    11648.664
2021-12-12 09:00:02    12087.144
                         ...    
2022-11-03 00:00:02     3275.496
2022-11-03 03:00:02      886.185
2022-11-03 09:00:02    12640.290
2022-11-03 12:00:02     6182.610
2022-11-03 21:00:02     1839.060
Length: 75, dtype: float64

In [89]:
states = [0]*len(proportions)
states

proportion_df = pd.DataFrame(proportions, columns = ["Proportions"])
proportion_df["State"] = states


In [90]:
# Assigning states by threshold
proportion_df.loc[proportion_df["Proportions"] == 0,"State"] = "State 1"
proportion_df.loc[proportion_df["Proportions"] <= threshold_state_1,"State"] = "State 1"
proportion_df.loc[(proportion_df["Proportions"] <= threshold_state_2) & (proportion_df["Proportions"] > threshold_state_1),"State"] = "State 2"
proportion_df.loc[(proportion_df["Proportions"] < 1) & (proportion_df["Proportions"] > threshold_state_3),"State"] = "State 3"
proportion_df.loc[proportion_df["Proportions"] == 1,"State"] = "State 4"

# example
proportion_df[proportion_df["State"] == "State 2"]




,Proportions,State
datetime,,
2022-03-24 00:00:02,0.614477,State 2
2022-08-19 21:00:02,0.531565,State 2
2022-08-20 03:00:02,0.492121,State 2
2022-11-03 03:00:02,0.458041,State 2


In [87]:
import numpy as np

def createStateDict():
    stringToIndex = {}
    for i in range(number_of_states):
        key = "State " + str(i+1)
        if not key in stringToIndex.keys():
            stringToIndex[key] = i        
    return stringToIndex

def getIndexByState(stateDict, state): 
    # for proportion == 0, state should got to 1. DataFrame cannot handle the "==" assignment, therefore it is manually assigned here.
    if state == 0:
        return 1
    else:
        return stateDict[state]


# calculate transitions
prev_index = 0
cur_row_index = 1

transition_probs = np.zeros((number_of_states,number_of_states))
transitions = np.zeros((number_of_states,number_of_states))
total_transitions_for_each_state = np.zeros(4)
state_dict = createStateDict()

for index, cur_row in proportion_df.iterrows():
  
    cur_index = getIndexByState(state_dict, cur_row["State"])
    transitions[prev_index][cur_index] += 1
    prev_index = cur_index
    total_transitions_for_each_state[prev_index] += 1

for i in range(number_of_states):
    transition_probs[i] = transitions[i]/total_transitions_for_each_state[i]


print("Transition probabilities: ")
print(transition_probs)
print()
# All state transitions are summed up to be 1
print(transition_probs[0,:].sum())
print(transition_probs[1,:].sum())
print(transition_probs[2,:].sum())
print(transition_probs[3,:].sum())






Transition probabilities: 
[[9.96128124e-01 2.81590989e-03 7.03977473e-04 3.51988736e-04]
 [1.87500000e-01 1.25000000e-01 1.25000000e-01 5.62500000e-01]
 [7.27272727e-01 1.81818182e-01 9.09090909e-02 0.00000000e+00]
 [0.00000000e+00 8.69565217e-02 1.30434783e-01 7.82608696e-01]]

1.0
1.0
1.0
1.0


array([[9.96128124e-01, 2.81590989e-03, 7.03977473e-04, 3.51988736e-04],
       [1.87500000e-01, 1.25000000e-01, 1.25000000e-01, 5.62500000e-01],
       [7.27272727e-01, 1.81818182e-01, 9.09090909e-02, 0.00000000e+00],
       [0.00000000e+00, 8.69565217e-02, 1.30434783e-01, 7.82608696e-01]])